# Introduction
### The data set is very messy with no exploratory name of attributes, so I add names on top of ever columns in according to the website in which data came from. The name of 26 attributes I added from left to right are: symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression,horsepower, peak-rpm,city-mpg,highway-mpg, price.  

#### Symboling corresponds to the degree to which the auto is more risky than its price indicates. Cars are initially assigned a risk factor symbol associated with its price. Then, if it is more risky (or less), this symbol is adjusted by moving it up (or down) the scale. In  the dataset, it means the auto which has symboling 3 has highest risk while the auto which has symboling -3 has the lowest risk.

#### Normalized-losses is the relative average loss payment per insured vehicle year. This value is normalized for all autos within a particular size classification, and represents the average loss per car per year. The values range from 65 to 256. 

#### A car's make is the brand of the vehicle

#### A motor fuel is a fuel that is used to provide power to motor vehicles.

#### A naturally-aspirated engine, also known as a normally-aspirated engine, is an internal combustion engine in which oxygen intake depends solely on atmospheric pressure and does not have forced induction through a turbocharger or a supercharger. The aspiration with turbo means there is a forced induction device in auto that increases an internal combustion engine's efficiency and power output by forcing extra compressed air into the combustion chamber.

#### The fuel system is made up of the fuel tank, pump, filter, and injectors or carburetor, and is responsible for delivering fuel to the engine as needed.

#### The bore  is the diameter of each cylinder.

#### Horsepower is a unit of power used to measure the forcefulness of a vehicle's engine. The total number of miles the vehicle can go during its lifespan is also determined using horsepower.

#### RPM stands for “Revolutions per Minute.” It’s a way of measuring the speed at which the engine revolves or spins.

#### City MPG refers to driving with occasional stopping and braking, simulating the conditions you're likely to run into while driving on city streets. Miles per gallon (mpg) is the measurement of the distance in miles that a vehicle can travel using only one gallon of fuel. 

### In this project, city-mpg is the responsive variable which I want to predict. I try to predict city-mpg using linear regression model, it is my job to choose what exploratory variables are regressors. 

### The data has plenty of missing values, so I plan to try some methods to deal with missing values.

### To simpliy the data without losing too much information, I did feature interaction by multiplying 'width' 'length' 'height' together as 'volume'.

### I plotted some graphics to see the linear relationships between city-mpg and other attributes, to see which pairs have more possibility to have linear relationship which could be used as linear model to predict city-mpg.

### I also checked the correlations between the regressors I choose to use. Since the multicollinearity exists, I choose to apply PCA on the sampling data, and since the regressors are not continuous, I choose to apply logistic regression model to make the prediction, and check the probability score of my logistic regression model to decide which regressors are the best for the model.


In [376]:
#import the file and important packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns

import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from numpy import linalg as LA
from sklearn.experimental import (
    enable_iterative_imputer,
)
from statsmodels.distributions.empirical_distribution  import ECDF
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
seed = 123

import pandas as pd
data = pd.read_excel("M4_Data.xls")

## Data preprocessing

#### To begin my data prepocessing, first I want to check what the types of data are in different features, check what are categorical variables, what are numeric variables, what type of numeric values are in numeric variables:

In [ ]:
 #check the types of data 
data.dtypes

#### After checking data types, I found there are 11 objects, which are categorical attributes, and there are 11 float type and 4 int type, which are numeric attributes. 26 attributes in all.

In [ ]:
#Check the data shape
print(data.shape)
data.head(5)

#### After checking the data shape, I found there are 205 rows and 26 columns in the whole data set, which means there are 205*26 = 5330 datas in whole, it is pretty a lot.

#### Next step, I will check whether there are mistakes and omissions on categorical variables:

In [ ]:
#Check whether there are mistakes and omissions on categorical variables.
classes = ['make','fuel-type','aspiration','num-of-doors','body-style','drive-wheels','engine-location','engine-type','num-of-cylinders','fuel-system']

for each in classes:
    print(each + ':\n')
    print(list(data[each].drop_duplicates()))
    print('\n')

#### The result shows there are no mistakes and omissions on categorical variables.

In [ ]:
#Do a descriptive statistics check on data
data_desc = data.describe()
data_desc

## After descriptive statistic checking on data, I found:

#### 1.There are 205 counts in symboling, wheel-base, length, width, height,curb-weight,engine-size,compression,city-mpg, and 203 counts in horsepower,peak-rpm, 201 counts in bore, stroke, price, and 164 counts in normalized-losses, from these information, I realized that the attribute normalized-losses have the highest number of missing values, and bore, stroke, price rank second. When dealing with missing data, the attribute normalized-losses should be put in the most important one.
#### 2. Symboling has the lowest mean, price has the highest mean value.
#### 3. Price has the largest standard deviation, while bore has the smallest standard deviation. 
#### 4. Price has the highest min value and symboling has the lowest min value, while price has the highest max value while symboling has the lowest max value.¶

#### From descriptive statistic checking, I found there are 6 attributes in numeric attributes which have missing values: horsepower, peak-rpm, bore, stroke, price and normalized-losses. Horsepower and peak-rpm have 2 missing values, bore, stroke and price have 4 missing values, while normalized-losses have 41 missing values.

#### To double check the missing value, I apply the code in the following to see how much proportion the missing values owned in comparison with the whole number of data in each specific attribute which has missing values:

In [ ]:
null_cols = ['normalized-losses','num-of-doors','bore','stroke','horsepower','peak-rpm','price']
total_rows = data.shape[0]
for each_col in null_cols:
    print('{}:{}'.format(each_col,data[pd.isnull(data[each_col])].shape[0]/total_rows))

#### The result shows in normalized-losses attribute, the missing values owned 20% of the whole data. Bore, stroke and price have around 1.9512% missing values, while num-of-doors, horsepower and peak-rpm have around 0.9756% missing values.

#### Data we were dealing with have both numerical and categorical data:
#### 1. In numerical data missing values can be replaced with the “average” of the values that the columns have.
#### 2. While in categorical data missing value can be handled with “Mode” (frequently occurring) value.

### 1. Listwise or case deletion: 
#### the most common approach to the missing data is to simply omit those cases with the missing data and analyze the remaining data. In num-of-doors, horsepower and peak-rpm, the missing values occupy less than 1% of the whole data, though price has around 1.9512% missing values, but price has little predictale value in predicting city-mpg in reason, therefore, I choose to delete missing values in price, num-of-doors, horsepower and peak-rpm.

In [377]:
data = data.replace('?',np.nan)
data.dropna(subset=['price'], axis=0,inplace=True)
data.dropna(subset=['horsepower'], axis=0,inplace=True)
data.dropna(subset=['peak-rpm'], axis=0,inplace=True)

### 2. Mean substitution: 
#### In a mean substitution, the mean value of a variable is used in place of the missing data value for that same variable. The theoretical background of the mean substitution is that the mean is a reasonable estimate for a randomly selected observation from a normal distribution. So let's check whether it is suitable to use mean substitution in normalized-losses, bore and stroke by checking their distributions in the following:

In [ ]:
#histogram
plt.subplot(122)
x = data['bore'].dropna()
sns.distplot(x,hist=True,kde=True,kde_kws={"color":"k","lw":3,"label":"KDE"},hist_kws={"histtype":"step","linewidth":3,"alpha":1,"color":"pink"})

In [ ]:
#histogram
plt.subplot(122)
x = data['stroke'].dropna()
sns.distplot(x,hist=True,kde=True,kde_kws={"color":"k","lw":3,"label":"KDE"},hist_kws={"histtype":"step","linewidth":3,"alpha":1,"color":"pink"})

In [ ]:
#histogram
plt.subplot(122)
x = data['normalized-losses'].dropna()
sns.distplot(x,hist=True,kde=True,kde_kws={"color":"k","lw":3,"label":"KDE"},hist_kws={"histtype":"step","linewidth":3,"alpha":1,"color":"pink"})

#### From the histogram I plotted, I could see, bore has two peaks in distribution, which does not satisfy the condition to be normal distribtuion, stroke is pretty fit in normal distribution, and normalized-losses is more closed to skew distribution. Based on the plot I make the conclusion, that I will fill in the missing value of stroke with mean value:

In [367]:
strokemean = data[['stroke']].mean()
strokemean
data[['stroke']] = data[['stroke']].replace(np.nan,strokemean)

### 3. Filling in missing data using KNN method: the advantage of KNN:

#### When dealing with filling in missing values of normalized-losses, I noticed normalized-losses is a skew distribution, so I chose to use KNN imputer to fill in missing data:

In [397]:
df = data['normalized-losses'].values.reshape(-1, 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df = scaler.transform(df)
imputer = KNNImputer(n_neighbors=2)
data['normalized-losses'] = imputer.fit_transform(df)

#### For bore, I also use KNN imputer to fill in missing values:

In [398]:
df = data['bore'].values.reshape(-1, 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df = scaler.transform(df)
imputer = KNNImputer(n_neighbors=2)
data['bore'] = imputer.fit_transform(df)

## Turning categorical variables into quantitative variables:

#### Dummy variable is a numerical variable used to label categories. They are called ‘dummies’ because the numbers themselves don’t have inherent meaning. In the auto data-set, the “fuel-type” feature as a categorical variable has two values, “gas” or “diesel”, which are in String format. For further analysis, it has to convert into some form of numeric format. In the case where the feature “Fuel” has two unique values, gas and diesel, we create two new features ‘gas’ and ‘diesel.’ 

In [ ]:
dummy_fueltype = pd.get_dummies(data['fuel-type'])
dummy_fueltype.rename(columns={'fuel-type':'gas','fuel-type':'diesel'},inplace = True)
data = pd.concat([data,dummy_fueltype],axis = 1)
data.drop('fuel-type',axis = 1, inplace = True)

#### In the auto data-set, the “aspiration” feature as a categorical variable has two values, “std” or “turbo”, which are in String format. For further analysis, it has to convert into some form of numeric format. In the case where the feature “Aspiration” has two unique values, std and turbo, we create two new features ‘std’ and ‘turbo.’ 

In [300]:
dummy_aspiration = pd.get_dummies(data['aspiration'])
dummy_aspiration.rename(columns={'aspiration':'std','aspiration':'turbo'},inplace = True)
data = pd.concat([data,dummy_aspiration],axis = 1)
data.drop('aspiration',axis = 1, inplace = True)

#### There are some features which have little significance for predictiong the city-mpg, such that: price, number of doors, engine location, so I decide to delete them:

In [ ]:
data = data.drop('price', axis=1)
data = data.drop('num-of-doors', axis=1)
data = data.drop('engine-location', axis=1)

## EDA(Exploratory Data Analysis )
### Exploratory Data Analysis,is an approach to analyze data in order to:
#### 1. summarize main characteristics of the data
#### 2. gain better understanding of the data-set,
#### 3. uncover relationships between different variables, and
#### 4. extract important variables for the problem we are trying to solve

### The question here is to find “What are the characteristics that have the most impact on the city-mpg?”
##### We just go through a couple of different useful exploratory data analysis techniques in order to answer this question.

In [ ]:
#Do a descriptive statistics check on data
data_desc = data.describe()
data_desc

## Summarize the categorical data by using the function value_counts().

In [ ]:
drive_wheel_count = data['drive-wheels'].value_counts()
drive_wheel_count

#### There are 116 Front-wheel drives, 75 rear-wheel drives, 8 four-wheel drives.

In [ ]:
body_style_count = data['body-style'].value_counts()
body_style_count

#### There are 94 sedan, 67 hatchback, 24 wagons, 8 hardtop and 6 convertibles body style.

In [ ]:
engine_type_count = data['engine-type'].value_counts()
engine_type_count

#### There are 143 ohc, 15 ohcf, 13 ohcv, 12 dohc, 12 l and 4 rotor of engine-type

In [ ]:
numberof_cylinders_count = data['num-of-cylinders'].value_counts()
numberof_cylinders_count

#### There are 155 four cylinders, 24 six cylinders, 10 five cylinders, 4 eight cylinders, 4 two cylinders, 1 three cylinders and 1 twelve cylinders

In [ ]:
fuel_system_count = data['fuel-system'].value_counts()
fuel_system_count

#### There are 90 mpfi, 64 2bbl, 20 idi, 11 lbbl, 9 spdi, 3 4bbl, 1 spfi and 1 mfi

## Boxplot
### Boxplots are a great way to visualize numeric data,the boxplot shows the “median” of the data, which represents where the middle datapoint is. The Upper Quartile shows where the ”75th percentile” is, the Lower Quartile shows where the “25th percentile” is. The data between the Upper and Lower Quartile represents the Interquartile Range(IQR). There are also Lower and Upper Extremes. These are calculated as 1.5 times the interquartile range above the 75th percentile, and as 1.5 times the IQR below the 25th percentile.
### Boxplots also display “outliers” as individual dots that occur outside the upper and lower extremes. With boxplots, I can easily spot outliers and also see the distribution and skewness of the data.

In [ ]:
sns.boxplot(x='drive-wheels',y='city-mpg',data = data)

#### From boxplot with x-axis is drive-wheels and y-axis is city-mpg, I make a small conclusion that the auto has front-wheel-drives have highest mean of city-mpg, while the auto has rear-wheel-drives have lowest mean of city-mpg.

In [ ]:
sns.boxplot(x='engine-type',y='city-mpg',data = data)

#### From boxplot with x-axis is engine-type and y-axis is city-mpg, I make a small conclusion that the auto has ohc have highest mean of city-mpg, while the auto has rotor and ohcv have lowest mean of city-mpg.

In [ ]:
sns.boxplot(x='body-style',y='city-mpg',data = data)

#### From boxplot with x-axis is body-style and y-axis is city-mpg, I make a small conclusion that the auto is hatchback has highest mean of city-mpg, while the auto is convertible has lowest mean of city-mpg.

In [ ]:
sns.boxplot(x='fuel-system',y='city-mpg',data = data)

#### From boxplot with x-axis is fuel-system and y-axis is city-mpg, I make a small conclusion that the auto is 2bbl has highest mean of city-mpg, while the auto is 4bbl has lowest mean of city-mpg.

In [ ]:
sns.boxplot(x='symboling',y='city-mpg',data = data)

#### From boxplot with x-axis is symboling and y-axis is city-mpg, I make a small conclusion that the auto has symboling 2 has highest mean of city-mpg, while the auto has symboling 3 has lowest mean of city-mpg.

## Feature extraction:
### In the dataset, I found there are three features: length, width and height which could be extracted as volume just by multiplying length, width and height together, therefore, I will do feature extractiom in the following code:

In [318]:
data['volume']  = data.length * data.width * data.height
data.drop(['width','length','height','curb-weight'],axis = 1, inplace = True)

### Then I want to make a boxplot with mixed features to get better macro visualization:
#### Since volume, peak-rpm, normalized-losses, engine-size, wheel-base and horsepower have very large number, for better visualization, I kicked out volume and peak-rpm in the features:

In [ ]:
features = data.drop(columns=['city-mpg','volume','peak-rpm','normalized-losses','engine-size','horsepower','wheel-base','gas','diesel','turbo','std'])
features.plot.box(title='Autocar',vert=False)
plt.xticks(rotation = -20)
plt.figure()
sns.boxplot(data = features, linewidth = 4, orient = 'h')

In [ ]:
features = data.drop(columns=['city-mpg','volume','peak-rpm','turbo','std','gas','diesel','turbo','std','horsepower','wheel-base','compression','stroke','bore','symboling'])
features.plot.box(title='Autocar',vert=False)
plt.xticks(rotation = -20)
plt.figure()
sns.boxplot(data = features, linewidth = 4, orient = 'h')

### From the boxplot above, we could see compression, stroke and engine size have more outliers.

## Scatterplot

#### The predictor variable: is the variable I am using to predict an outcome. 
#### The target variable: is the variable I am trying to predict. 

### In a scatterplot, we typically set the predictor variable on the x-axis, or horizontal axis and we set the target variable on the y-axis or vertical axis.

### Let's plot plenty of scatter point to see which features have more tendency to have better linear relationship with city-mpg.
 
#### I will do scatter plot with features in the following sequence: symboling, normalized-losses, wheel-base, volume, curb-weight, engine-size, bore, stroke, compression, horsepower, peak-rpm, highway-mpg. 

In [ ]:
x = data['symboling']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between symboling and city-mpg')
plt.xlabel('symboling')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between symboling and city-mpg manifest there is hardly a linear relationship between them.

In [ ]:
x = data['normalized-losses']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between normalized-losses and city-mpg')
plt.xlabel('normalized-losses')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between normalized-losses and city-mpg manifest there is hardly a linear relationship between them.

In [ ]:
x = data['wheel-base']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between wheel-base and city-mpg')
plt.xlabel('wheel-base')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between wheel-base and city-mpg manifest there may be a negative linear relationship between them, but the linear model is not so apparently.

In [ ]:
x = data['volume']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between volume and city-mpg')
plt.xlabel('volume')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between volume and city-mpg manifest there may be a negative linear relationship between them, but the points is a little disparse for a linear model.

In [ ]:
x = data['engine-size']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between engine-size and city-mpg')
plt.xlabel('engine-size')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between engine-size and city-mpg manifest there may be a linear relationship between them, with some outliers.

In [ ]:
x = data['bore']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between bore and city-mpg')
plt.xlabel('bore')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between bore and city-mpg manifest there may be a linear relationship between them, but the points are disparse, and outliers exist.

In [ ]:
x = data['stroke']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between stroke and city-mpg')
plt.xlabel('stroke')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between stroke and city-mpg manifest there is hardly a linear relationship between them.

In [ ]:
x = data['compression']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between compression and city-mpg')
plt.xlabel('compression')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between compression and city-mpg manifest there is hardly a linear relationship between them.

In [ ]:
x = data['horsepower']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between horsepower and city-mpg')
plt.xlabel('horsepower')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between horsepower and city-mpg manifest there may be a linear relationship between them.

In [ ]:
x = data['peak-rpm']
y = data['city-mpg']
plt.scatter(x,y)
plt.title('Scatter between peak-rpm and city-mpg')
plt.xlabel('peak-rpm')
plt.ylabel('city-mpg')
plt.show()

### The scattor plot between peak-rpm and city-mpg manifest there is hardly a linear relationship between them.

#### Also, without checking, city-mpg and highway-mpg must have linear relationship in reason.

### From scatter plot above, we know that the possible linear relatioship pairs are: wheel-base and city-mpg, volume and city-mpg, engine-size and city-mpg, bore and city-mpg, horsepower and city-mpg, highway-mpg and city-mpg. In these pairs, horsepower and city-mpg, highway-mpg and city-mpg are most appropriate for linear regression model.

## Gropuby
### The “groupby” method groups data by different categories. The data is grouped based on one or several variables and analysis is performed on the individual groups.

In [124]:
data_group = data[['wheel-base','volume','city-mpg']]
grouped_avg = data_group.groupby(['wheel-base','volume'],as_index=False).mean()
pivot_table = grouped_avg.pivot(index = 'wheel-base',columns = 'volume')

## It is a great way to plot the target variable over multiple variables and through heatmap get visual clues of the relationship between these variables and the target.

In [ ]:
sns.heatmap(pivot_table)
plt.show()

## Analysis of Variance (ANOVA)
### The Analysis of Variance (ANOVA) is a statistical method used to test whether there are significant differences between the means of two or more groups. ANOVA returns two parameters:
#### F-test score: ANOVA assumes the means of all groups are the same, calculates how much the actual means deviate from the assumption, and reports it as the F-test score. A larger score means there is a larger difference between the means.
#### P-value: P-value tells how statistically significant is our calculated score value

In [ ]:
group = data[['make','city-mpg']]
group_anova = group.groupby(['make'],as_index = False)
from scipy import stats
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('audi')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('bmw')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('chevrolet')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('dodge')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('honda')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('isuzu')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('jaguar')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('mazda')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('mercedes-benz')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('mitsubishi')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('nissan')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('peugot')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('plymouth')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('porsche')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('saab')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('subaru')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('toyota')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('volkswagen')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('volvo')['city-mpg'])
anova_result

In [ ]:
anova_result = stats.f_oneway(group_anova.get_group('alfa-romero')['city-mpg'],group_anova.get_group('mercury')['city-mpg'])
anova_result

### I extract the make and city-mpg data. Then I group the data by different makes. 
#### The anovo_result says city-mpg between alfa-romero and audi are not significantly different, since the F-score is less than 1 and the p-value is larger than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and bmw are not significantly different, since the F-score is less than 1 and the p-value is larger than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and chevrolet are not significantly different, since the F-score is very large but the p-value is less than 0.05.
#### The anovo_result says city-mpg between alfa-romero and dodge are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and honda are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and isuzu are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and jaguar significantly different, since the F-score is very large but the p-value is very small.
#### The anovo_result says city-mpg between alfa-romero and mazda are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and mercedes-benz are very significantly different, since the F-score is less than 1 and the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and mitsubishi are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and nissan are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and peugot are very significantly different, since the F-score is less than 1 and the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and plymouth are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and porsche are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and saab are significantly different, since the F-score is very small and the p-value is larger than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and subaru are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and toyota are significantly different, since the F-score is greater than 1 but the p-value is larger than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and volkswagen are significantly different, since the F-score is greater than 1 but the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and volvo are very significantly different, since the F-score is less than 1 and the p-value is less than 0.05. 
#### The anovo_result says city-mpg between alfa-romero and mercury are very significantly different, since the F-score is less than 1 and the p-value is larger than 0.05. 


## In most cases, different makes have different city-mpg with alfa-romero, which means 'make' takes a very important role in diifference of city-mpg.

## Correlation

### To do multiple linear regression model, it is very important to check whether there are multicollinearities between dfferent independent variables, therefore, it is crucial to check the correlations between feautures which are potential to be the regressors of the linear regression model. 

In [ ]:
sns.regplot(x = 'wheel-base',y = 'volume',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a positive linear relationship between wheel-base and volume. With increase in values of wheel-base, volume goes up as well and the slope of the line is positive, so there is a positive correlation between “wheel-base” and "volume”.

In [ ]:
sns.regplot(x = 'wheel-base',y = 'engine-size',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a weak positive linear relationship between wheel-base and engine-size. With increase in values of wheel-base, engine-size goes up as well and the slope of the line is positive, so there is a positive correlation between “wheel-base” and "engine-size”.

In [ ]:
sns.regplot(x = 'wheel-base',y = 'bore',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a very weak positive linear relationship between wheel-base and bore. 

In [ ]:
sns.regplot(x = 'wheel-base',y = 'horsepower',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a weak positive linear relationship between wheel-base and horsepower. With increase in values of wheel-base, horsepower goes up as well and the slope of the line is positive, so there is a positive correlation between “wheel-base” and "horsepower”.

In [ ]:
sns.regplot(x = 'wheel-base',y = 'highway-mpg',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a weak negative linear relationship between wheel-base and highway-mpg. With increase in values of wheel-base, highway-mpg goes down as well and the slope of the line is negative, so there is a negative correlation between “wheel-base” and "highway-mpg”.

In [ ]:
sns.regplot(x = 'volume',y = 'engine-size',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a strong positive linear relationship between volume and engine-size. With increase in values of volume, engine-size goes up as well and the slope of the line is positive, so there is a positive correlation between “volume” and "engine-size”.

In [ ]:
sns.regplot(x = 'volume',y = 'bore',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a positive linear relationship between volume and bore. With increase in values of volume, bore goes up as well and the slope of the line is positive, so there is a positive correlation between “volume” and "bore”.

In [ ]:
sns.regplot(x = 'volume',y = 'horsepower',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a positive linear relationship between volume and horsepower. With increase in values of volume, horsepower goes up as well and the slope of the line is positive, so there is a positive correlation between “volume” and "horsepower”.

In [ ]:
sns.regplot(x = 'volume',y = 'highway-mpg',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a weak negative linear relationship between volume and highway-mpg. With increase in values of volume, highway-mpg goes down as well and the slope of the line is negative, so there is a weak negative correlation between “volume” and "highway-mpg”.

In [ ]:
sns.regplot(x = 'engine-size',y = 'bore',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a positive linear relationship between engine-size and bore. With increase in values of engine-size, bore goes up as well and the slope of the line is positive, so there is a positive correlation between “engine-size” and "bore”.

In [ ]:
sns.regplot(x = 'engine-size',y = 'horsepower',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a positive linear relationship between engine-size and horsepower. With increase in values of engine-size, horsepower goes up as well and the slope of the line is positive, so there is a positive correlation between “engine-size” and "horsepower”.

In [ ]:
sns.regplot(x = 'engine-size',y = 'highway-mpg',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a weak negative linear relationship between engine-size and highway-mpg. With increase in values of engine-size, highway-mpg goes down as well and the slope of the line is negative, so there is a negative correlation between “engine-size” and "highway-mpg”.

In [ ]:
sns.regplot(x = 'bore',y = 'horsepower',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a weak positive linear relationship between bore and horsepower. With increase in values of bore, horsepower goes up as well and the slope of the line is positive, so there is a positive correlation between “bore” and "horsepower”.

In [ ]:
sns.regplot(x = 'bore',y = 'highway-mpg',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a very weak negative linear relationship between bore and highway-mpg. With increase in values of bore, highway-mpg goes down as well and the slope of the line is negative, so there is a negative correlation between “bore” and "highway-mpg”.

In [ ]:
sns.regplot(x = 'horsepower',y = 'highway-mpg',data = data)
plt.ylim(0,)
plt.show()

#### I see that the straight line shows that there is a negative linear relationship between horsepower and highway-mpg. With increase in values of horsepower, highway-mpg goes down as well and the slope of the line is negative, so there is a negative correlation between “horsepower” and "highway-mpg”.

## From above, we could pick the weak correlation pairs: wheel-base and engine-size, wheel-base and bore, wheel-base and horsepower, wheel-base and highway-mpg, volume and highway-mpg, engine-size and highway-mpg, bore and horsepower, bore and highway-mpg, horsepower and highway-mpg could be suitable regressors for linear regression model.

## Pearson Correlation
### The Pearson Correlation measures the linear dependence between two variables X and Y. The resulting coefficient is a value between -1 and 1 inclusive, where:
#### 1: total positive linear correlation,
#### 0: no linear correlation, the two variables most likely do not affect each other
#### -1: total negative linear correlation.

#### The P-value is the probability value that the correlation between these two variables is statistically significant. Normally, we choose a significance level of 0.05, which means that we are 95% confident that the correlation between the variables is significant.
#### By convention, when the
##### p-value is < 0.001 we say there is strong evidence that the correlation is significant,
##### the p-value is < 0.05; there is moderate evidence that the correlation is significant,
##### the p-value is < 0.1; there is weak evidence that the correlation is significant, and
##### the p-value is > 0.1; there is no evidence that the correlation is significant.


In [ ]:
#Check the correlation of the data
cor_matrix = data.corr()
cor_matrix

### Let’s calculate the Pearson Correlation Coefficient and P-value of each pair.

In [ ]:
pearson_coef,p_value = stats.pearsonr(data['wheel-base'],data['engine-size'])
print("the Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P is",p_value)

#### Since the p-value is < 0.001, the correlation between wheel-base and engine-size is statistically significant, although the linear relationship isn’t extremely strong (~0.573)

In [ ]:
pearson_coef,p_value = stats.pearsonr(data['wheel-base'],data['bore'])
print("the Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P is",p_value)

#### Since the p-value is < 0.001, the correlation between wheel-base and bore is statistically significant, although the linear relationship isn’t extremely strong (~0.478)

In [ ]:
pearson_coef,p_value = stats.pearsonr(data['wheel-base'],data['horsepower'])
print("the Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P is",p_value)

#### Since the p-value is < 0.001, the correlation between wheel-base and horsepower is statistically significant, although the linear relationship isn’t extremely strong (~0.371)

In [ ]:
pearson_coef,p_value = stats.pearsonr(data['wheel-base'],data['highway-mpg'])
print("the Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P is",p_value)

#### Since the p-value is < 0.001, the correlation between wheel-base and highway-mpg is statistically significant, although the linear relationship isn’t extremely strong (~-0.542)

In [ ]:
pearson_coef,p_value = stats.pearsonr(data['volume'],data['highway-mpg'])
print("the Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P is",p_value)

#### Since the p-value is < 0.001, the correlation between volume and highway-mpg is statistically significant, although the linear relationship isn’t extremely strong (~-0.598)

In [ ]:
pearson_coef,p_value = stats.pearsonr(data['engine-size'],data['highway-mpg'])
print("the Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P is",p_value)

#### Since the p-value is < 0.001, the correlation between engine-size and highway-mpg is statistically significant, although the linear relationship isn’t extremely strong (~-0.680)

## As tested, all pairs have strong corrlations, which is not suitable to construct a linear regression model, therefore, I want to apply for PCA. 

### PCA is effected by scale so I need to scale the features in data before applying PCA. Use StandardScaler to help me standardize the dataset’s features onto unit scale (mean = 0 and variance = 1) which is a requirement for the optimal performance of machine learning algorithms. 

### Code projects the original data which is 5 dimensional into 2 dimensions. After dimensionality reduction, there usually isn’t a particular meaning assigned to each principal component. The new components are just the two main dimensions of variation.

In [194]:
from sklearn.preprocessing import StandardScaler
features = ['wheel-base', 'engine-size', 'bore', 'volume','horsepower']
# Separating out the features
x = data.loc[:, features].values
# Separating out the target
y = data.loc[:,['city-mpg']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

### Concatenating DataFrame along axis = 1. finalDf is the final DataFrame before plotting the data.

In [ ]:
finalDf = pd.concat([principalDf, data[['city-mpg']]], axis = 1)

## Visualize 2D Projection
### This section is plotting 2 dimensional data. 

In [202]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['17', '19', '20']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['city-mpg'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

## Explained Variance

In [ ]:
pca.explained_variance_ratio_

### By using the attribute explained_variance_ratio_, I can see that the first principal component contains 66.71% of the variance and the second principal component contains 18.37% of the variance. Together, the two components contain 85.08% of the information.

## PCA to Speed-up Machine Learning Algorithms

### Split Data into Training and Test Sets
#### In this case, I chose 80% of the data to be training and 20% of the data to be in the test set.

In [208]:
X = data[['wheel-base','engine-size','bore','volume','horsepower']]
Y = data['city-mpg']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)

### Standardize the Data

In [213]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

### Import and Apply PCA
#### Notice the code below has .95 for the number of components parameter. It means that scikit-learn choose the minimum number of principal components such that 95% of the variance is retained.

In [214]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)

### Fit PCA on training set.

In [ ]:
pca.fit(X_train)

### Apply the mapping (transform) to both the training set and the test set.

In [243]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

## Apply Logistic Regression to the Transformed Data

### Why do we use logistic regression other than regular linear regression: Because there are input variables with inconsistent numbers of samples.

### Properties of Logistic Regression:

#### 1.The dependent variable in logistic regression follows Bernoulli Distribution.
#### 2. Estimation is done through maximum likelihood.
#### 3. No R Square, Model fitness is calculated through Concordance, KS-Statistics.

### Linear Regression Vs. Logistic Regression
#### Linear regression gives you a continuous output, but logistic regression provides a constant output. 
#### Linear regression is estimated using Ordinary Least Squares (OLS) while logistic regression is estimated using Maximum Likelihood Estimation (MLE) approach.

#### Step 1: Import the model I want to use

In [251]:
from sklearn.linear_model import LogisticRegression

#### Step 2: Make an instance of the Model.

In [245]:
logisticRegr = LogisticRegression(solver = 'lbfgs')

#### Step 3: Training the model on the data, storing the information learned from the data

In [ ]:
logisticRegr.fit(X_train, Y_train)

#### Step 4: Predict new data 
#### Uses the information the model learned during the model training process
#### The code below predicts for one observation

In [ ]:
# Predict for One Observation
logisticRegr.predict(X_test[0].reshape(1,-1))

#### The code below predicts for multiple observations at once

In [ ]:
# Predict for One Observation
logisticRegr.predict(X_test[0:10])

## Measuring Model Performance
### Accuracy is used here for simplicity.

In [ ]:
logisticRegr.score(X_test, Y_test)

### The score is low with 0.225, which means the model does not provide a lot of valueable information for predicting, pick another sampling of regressors to see what happens:

#### I choose the regressors : wheel-base,engine-size, bore and volume, and check the logistic probability score again:

In [ ]:
X = data[['wheel-base','engine-size','bore','volume']]
Y = data['city-mpg']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(X_train, Y_train)
logisticRegr.predict(X_test[0].reshape(1,-1))
logisticRegr.predict(X_test[0:10])
logisticRegr.score(X_test, Y_test)

### The logistic probability score is 0.175

#### I choose the regressors : wheel-base,engine-size and bore, and check the logistic probability score again:

In [263]:
X = data[['wheel-base','engine-size','bore']]
Y = data['city-mpg']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(X_train, Y_train)
logisticRegr.predict(X_test[0].reshape(1,-1))
logisticRegr.predict(X_test[0:10])
logisticRegr.score(X_test, Y_test)

0.2

### The logistic probability score is 0.2

#### I choose the regressors : wheel-base and engine-size, and check the logistic probability score again:

In [ ]:
X = data[['wheel-base','engine-size']]
Y = data['city-mpg']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(X_train, Y_train)
logisticRegr.predict(X_test[0].reshape(1,-1))
logisticRegr.predict(X_test[0:10])
logisticRegr.score(X_test, Y_test)

### The logistic probability score is 0.225

#### I choose the regressors : wheel-base and horsepower, and check the logistic probability score again:

In [ ]:
X = data[['wheel-base','horsepower']]
Y = data['city-mpg']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(X_train, Y_train)
logisticRegr.predict(X_test[0].reshape(1,-1))
logisticRegr.predict(X_test[0:10])
logisticRegr.score(X_test, Y_test)

### The logistic probability score is 0.275

#### I choose the regressors : volume and horsepower, and check the logistic probability score again:

In [ ]:
X = data[['volume','horsepower']]
Y = data['city-mpg']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(X_train, Y_train)
logisticRegr.predict(X_test[0].reshape(1,-1))
logisticRegr.predict(X_test[0:10])
logisticRegr.score(X_test, Y_test)

### The logistic probability score is 0.3

### Many times of trying shows that choose 'volume' and 'horsepower' as features to do PCA will give the highest probability score of logistic regression with 0.3. But the score is still high, Let's try to check multicollinearity of features, then apply for linear regression model:

### Although correlation matrix and scatter plots can also be used to find multicollinearity, their findings only show the bivariate relationship between the independent variables. VIF is preferred as it can show the correlation of a variable with a group of other variables

In [351]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

X = data[['wheel-base','engine-size','bore','volume','horsepower']]
calc_vif(X)

,variables,VIF
0,wheel-base,465.186967
1,engine-size,38.852617
2,bore,227.252473
3,volume,226.408347
4,horsepower,31.120716


### From the experiment I can see that the ‘wheel-base’ and ‘bore’ and 'volume' have a high VIF value, meaning they can be predicted by other independent variables in the dataset.

### Therefore, I can choose engine-size and horsepower to do linear regression model:

In [ ]:
X = data[['wheel-base','bore']]
Y = data['city-mpg']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 10)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()  
regressor.fit(X_train, Y_train)

### To see the value of the intercept and slope calculated by the linear regression algorithm for our dataset, execute the following code:

In [ ]:
#To retrieve the intercept:
print(regressor.intercept_)
#For retrieving the slope:
print(regressor.coef_)

### The result is -1.95995437 and -2.80233908 respectively.

### We have trained our algorithm, it’s time to make some predictions and see how accurately our algorithm predicts the percentage score. 

In [359]:
Y_pred = regressor.predict(X_test)

### Now compare the actual output values for X_test with the predicted values, execute the following script:

In [ ]:
from sklearn import metrics
dataprediction = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
dataprediction

### We can also visualize comparison result as a bar graph using the below script :

In [362]:
df1 = dataprediction.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

### As we can observe here that our model has returned a good prediction results.
### The final step is to evaluate the performance of the algorithm. We’ll do this by finding the values for MAE, MSE, and RMSE. 

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

## Conclusion
### From the long experiment, I can make the conclusions as the follow:
### 1. In lots of features, engine size and horsepower has less multicollinearity comparing with other features.
### 2. All numeric features have correlations, which makes the linear model not the best in this case
### 3. When applying the PCA, horsepower also takes an important role, which means horsepower may be the most important indicator in predicting the city-mpg.
### The PCA logistic regression has low probability score, so I rejected the PCA logistic regression model.
### Though the Mean Absolute Error and Root Mean Squared Error is not as ideal as to be lower than 1, but ###  What I want is a balance between overfit (very low MSE for training data) and underfit (very high MSE for test/validation/unseen data). So the linear regression model was not very accurate but can still make reasonably good predictions.

### There are many factors that may have contributed to inaccuracy, for example :
#### Need more data: I need to have a huge amount of data to get the best possible prediction.
#### Bad assumptions: I made the assumption that this data has a linear relationship, but that might not be the case. 
#### Poor features: The features I used have had a high enough correlation to the values we were trying to predict.
#### Other reasons: As I have tested, different makes have different city-mpg, the categorical features may also affect the linear model but I did not take them into accounts.